In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
family_group = "8.Lycaenidae"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")) and not(file.endswith("_old_backup.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
#     print(coordinate_info_dictionary)
#     print(coordinate_detail_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Aricia_agestis
Exon_1 Match
[['Aricia_agestis', ['32775356', '32775504']], ['Danaus_plexippus', ['32775356', '32775504']], ['Maniola_jurtina', ['32775356', '32775504']]]
Exon_2 Match
[['Aricia_agestis', ['32771356', '32771504']], ['Danaus_plexippus', ['32771356', '32771504']], ['Maniola_jurtina', ['32771356', '32771504']]]
Exon_3 Match
[['Aricia_agestis', ['32771022', '32771203']], ['Danaus_plexippus', ['32771022', '32771203']], ['Maniola_jurtina', ['32771022', '32771203']]]
Exon_4 Match
[['Aricia_agestis', ['32769986', '32770103']], ['Danaus_plexippus', ['32769986', '32770103']], ['Maniola_jurtina', ['32769986', '32770103']]]
Exon_5 Match
[['Aricia_agestis', ['32765550', '32765645']], ['Danaus_plexippus', ['32765550', '32765645']], ['Maniola_jurtina', ['32765550', '32765645']]]
Exon_6 mismatch
[['Aricia_agestis', ['32763192', '32763330']], ['Danaus_plexippus', ['32763192', '32763330']], ['Maniola_jurtina', ['32770045', '32770192']]]
Proceed?y
Aricia_agestis Exon_6 160_frame1 0.0 
MLQY

Choose Species 1 - 31
Exon_7 Match
[['Aricia_agestis', ['35257816', '35258054']], ['Danaus_plexippus', ['35257816', '35258054']], ['Maniola_jurtina', ['35257816', '35258054']]]
Exon_8 Match
[['Aricia_agestis', ['35256139', '35256286']], ['Danaus_plexippus', ['35256139', '35256286']], ['Maniola_jurtina', ['35256139', '35256286']]]
Exon_9 Match
[['Aricia_agestis', ['35227952', '35228135']], ['Danaus_plexippus', ['35227952', '35228135']], ['Maniola_jurtina', ['35227952', '35228135']]]
Exon_10 mismatch
[['Aricia_agestis', ['35224167', '35224343']], ['Danaus_plexippus', ['35224167', '35224343']], ['Maniola_jurtina', ['35224167', '35224373']]]
Proceed?y
Aricia_agestis Exon_10 693_frame2 0.0 
PSSVKSTSGSTTGTPTASGSAAWSRVTSQVCYSESDRASMSSESRSSHRTNTRDECRSG
Danaus_plexippus Exon_10 693_frame2 1.403 
PSSVKSTSGSTTGTPTASGSAAWSRVTSQVCYSESDRASMSSESRSSHRTNTRDECRSG
Maniola_jurtina Exon_10 693_frame2 1.408 
SRLTNTSPFQPSSVKSTSGSTTGTPTASGSAAWSRVTSQVCYSESDRASMSSESRSSHRTNTRDECRSG
Check Alignment??y
CLUSTAL for

Choose Species 1 - 3n
Exon_11 Match
[['Aricia_agestis', ['182902', '183057']], ['Danaus_plexippus', ['182902', '183057']], ['Maniola_jurtina', ['182902', '183057']]]
Exon_12 Match
[['Aricia_agestis', ['185076', '185198']], ['Danaus_plexippus', ['185076', '185198']], ['Maniola_jurtina', ['185076', '185198']]]
Exon_13 Match
[['Aricia_agestis', ['186922', '187083']], ['Danaus_plexippus', ['186922', '187083']], ['Maniola_jurtina', ['186922', '187083']]]
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Calycopis_cecrops,LUGF01024967.1,161686,161834,0,N,Aricia_agestis_XM_042131827.1_Frame_0_rightoh_2_query_Exon_1,0,0,0
Calycopis_cecrops,LUGF01024967.1,165686,165834,0,N,Aricia_agestis_XM_042131827.1_Frame_1_rightoh_1_query_Exon_2,1,49,49,Y,N
Calycopis_cecrops,LUGF01024967.1,166279,166460,0,N,Aricia_agestis_XM_042131827.1_Frame_2_rightoh_0_query_Exon_3,1,60,60,Y,N
Calycopis_cecrops,LUGF01024967.1,167112,167232,0,N,Aricia_agestis_XM_042131827.1_Frame_0_right

Proceed?y
Aricia_agestis Exon_4 80_frame2 0.0 
NDIIQTNLFDLTYQDDRSNLYNILQNNTTAADSTVTQSA
Danaus_plexippus Exon_4 80_frame2 0.97 
NDIIQTNLFDLTYQDDRSNLYNILQNNTTAADSTVTQSA
Maniola_jurtina Exon_4 80_frame2 1.231 
NDIIQTNLFDLTYQDDRSNLYNILQNNTTAADSTVTQSAGKSSYYVIYIICLYCGLHSTFRLILPR
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ -----------------------------------------------NDIIQTNLFDLTY
set80_frame2    FEQKTRFIYLAVTVCQIVSPTYKTNTATLIRPVLSFFINKTLRTLFQNDIIQTNLFDLTY
                                                               *************

Aricia_agestis_ QDDRSNLYNILQNNTTAADSTVTQSA----------------------------------
set80_frame2    QDDRSNLYNILQNNTTAADSTVTQSAGKSSYYVIYIICLYCGLHSTFRLILPRSIHERDS
                **************************                                  

Aricia_agestis_ ------------
set80_frame2    YCVYIIFQYKVK
                            

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Danaus_plexippu ---------------------------------

Exon_1 Match
[['Aricia_agestis', ['628101', '628249']], ['Danaus_plexippus', ['628101', '628249']], ['Maniola_jurtina', ['628101', '628249']]]
Exon_2 Match
[['Aricia_agestis', ['632101', '632249']], ['Danaus_plexippus', ['632101', '632249']], ['Maniola_jurtina', ['632101', '632249']]]
Exon_3 Match
[['Aricia_agestis', ['632678', '632859']], ['Danaus_plexippus', ['632678', '632859']], ['Maniola_jurtina', ['632678', '632859']]]
Exon_4 mismatch
[['Aricia_agestis', ['633127', '633247']], ['Danaus_plexippus', ['633127', '633247']], ['Maniola_jurtina', ['633127', '633289']]]
Proceed?y
Aricia_agestis Exon_4 22_frame0 0.756 
SDVINTNLFDLIFQDDRQKLYNILQNSVPVADSNRLLASP
Danaus_plexippus Exon_4 22_frame0 1.046 
SDVINTNLFDLIFQDDRQKLYNILQNSVPVADSNRLLASP
Maniola_jurtina Exon_4 22_frame0 1.332 
SDVINTNLFDLIFQDDRQKLYNILQNSVPVADSNRLLASPGWKTLYHYGLVIFL
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ --------------NDIIQTNLFDLTYQDDRSNLYNILQNNTTAADSTVTQSA-------
set22_fra

...

Exon_1 Match
[['Aricia_agestis', ['15503293', '15503441']], ['Danaus_plexippus', ['15503293', '15503441']], ['Maniola_jurtina', ['15503293', '15503441']]]
Exon_2 Match
[['Aricia_agestis', ['15499293', '15499441']], ['Danaus_plexippus', ['15499293', '15499441']], ['Maniola_jurtina', ['15499293', '15499441']]]
Exon_3 Match
[['Aricia_agestis', ['15492568', '15492749']], ['Danaus_plexippus', ['15492568', '15492749']], ['Maniola_jurtina', ['15492568', '15492749']]]
Exon_4 mismatch
[['Aricia_agestis', ['15489012', '15489129']], ['Danaus_plexippus', ['15489012', '15489129']], ['Maniola_jurtina', ['15488997', '15489129']]]
Proceed?y
Aricia_agestis Exon_4 203_frame1 0.455 
NDIVNTNLFDLTFQEDRANLYNILQNNSTATDSSLLASP
Danaus_plexippus Exon_4 203_frame1 0.93 
NDIVNTNLFDLTFQEDRANLYNILQNNSTATDSSLLASP
Maniola_jurtina Exon_4 203_frame1 1.187 
NDIVNTNLFDLTFQEDRANLYNILQNNSTATDSSLLASPGKSSY
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ ----------------------NDIIQTNLFD

Choose Species 1 - 31
Exon_5 Match
[['Aricia_agestis', ['7592983', '7593078']], ['Danaus_plexippus', ['7592983', '7593078']], ['Maniola_jurtina', ['7592983', '7593078']]]
Exon_6 mismatch
[['Aricia_agestis', ['7599165', '7599246']], ['Danaus_plexippus', ['7599114', '7599246']], ['Maniola_jurtina', ['7599165', '7599234']]]
Proceed?y
Aricia_agestis Exon_6 196_frame1 0.127 
MLQYIDQSDNEEHSGYQREQESRQVF
Danaus_plexippus Exon_6 196_frame1 1.256 
IVKNLCLDLNFCIIFLGMLQYIDQSDNEEHSGYQREQESRQVF
Maniola_jurtina Exon_6 102_frame2 1.266 
EEC
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ --------------------------------MLQYVEQTDNEEHSGYQREQES------
set196_frame1   PPFLKSVKNIILYLVIVKNLCLDLNFCIIFLGMLQYIDQSDNEEHSGYQREQESRQVFLY
                                                ****::*:**************      

Aricia_agestis_ -------
set196_frame1   LLSKFSY
                       

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Danaus_plexippu -----------------KNL---

Exon_1 Match
[['Aricia_agestis', ['30249252', '30249400']], ['Danaus_plexippus', ['30249252', '30249400']], ['Maniola_jurtina', ['30249252', '30249400']]]
Exon_2 Match
[['Aricia_agestis', ['30245252', '30245400']], ['Danaus_plexippus', ['30245252', '30245400']], ['Maniola_jurtina', ['30245252', '30245400']]]
Exon_3 Match
[['Aricia_agestis', ['30244815', '30244996']], ['Danaus_plexippus', ['30244815', '30244996']], ['Maniola_jurtina', ['30244815', '30244996']]]
Exon_4 Match
[['Aricia_agestis', ['30244289', '30244406']], ['Danaus_plexippus', ['30244289', '30244406']], ['Maniola_jurtina', ['30244289', '30244406']]]
Exon_5 Match
[['Aricia_agestis', ['30242242', '30242337']], ['Danaus_plexippus', ['30242242', '30242337']], ['Maniola_jurtina', ['30242242', '30242337']]]
Exon_6 mismatch
[['Aricia_agestis', ['30238887', '30239064']], ['Danaus_plexippus', ['30238887', '30239064']], ['Maniola_jurtina', ['30238995', '30239064']]]
Proceed?y
Aricia_agestis Exon_6 114_frame1 0.085 
MLQYIEQTDNEEHSGYQ

Choose Species 1 - 31
Exon_10 mismatch
[['Aricia_agestis', ['111005', '111256']], ['Danaus_plexippus', ['107700', '107921']], ['Maniola_jurtina', ['111005', '111211']]]
Proceed?y
Aricia_agestis Exon_10 116_frame0 1.713 
ITGEDTLSKPLSQLLMRRQWYATQACKSETKHQTVSDVKPLLQPWYATQPCKSETKHQTVSDVKPVLQPWYATQPCKSETKHQT
Danaus_plexippus Exon_10 44_frame1 1.697 
LRSLLRILKMCSLFSYADVVKSAMEDNTKTESANDVEQSQTMMNTTSSEDVIASVASSYIQENNDAFNTYQSHE
Maniola_jurtina Exon_10 116_frame0 1.773 
ITGEDTLSKPLSQLLMRRQWYATQACKSETKHQTVSDVKPLLQPWYATQPCKSETKHQTVSDVKPVLQP
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ PS------SVKSTSGSTTGTPTASGSAAWSRVTSQVCYSESDRASMSS------------
set116_frame0   CGYVYFIFNLKITGEDTLSKPLSQLLMRRQWYATQACKSETKHQTVSDVKPLLQPWYATQ
                 .      .:* *. .* ..* :.     .  ::*.* **:.: ::*.            

Aricia_agestis_ --ESRSSHRT-------------------NTRDE--------------------------
set116_frame0   PCKSETKHQTVSDVKPVLQPWYATQPCKSETKHQTVSDLKPVLQPWCATQPCKSENKHQT
        

Choose Species 1 - 3n
Exon_5 Match
[['Aricia_agestis', ['612261', '612359']], ['Danaus_plexippus', ['612261', '612359']], ['Maniola_jurtina', ['612261', '612359']]]
Exon_6 mismatch
[['Aricia_agestis', ['613055', '613178']], ['Danaus_plexippus', ['0', '0']], ['Maniola_jurtina', ['613112', '613178']]]
Proceed?y
Aricia_agestis Exon_6 320_frame1 1.275 
QNICKSPMLIFLMVVFFTGTNVDQTDSDDGSSTCQRDQES
Danaus_plexippus Exon_6 113_frame1 1.373 
ENFTGRQSRL
Maniola_jurtina Exon_6 320_frame1 1.242 
TNVDQTDSDDGSSTCQRDQES
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Aricia_agestis_ ---------------MLQY----------VEQTDNEEHSGY-QREQES---------
set320_frame1   SRNSATIVQNICKSPMLIFLMVVFFTGTNVDQTDSDDGSSTCQRDQESRYLILIIKS
                               ** :          *:***.:: *.  **:***         

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Danaus_plexippu --KNLESNENGHHSYQDEHES
set113_frame1   SLENLEENFTGRQS-RLKLLV
                  :***.* .*::* : :   

CLUSTAL format alignment by M